<a href="https://colab.research.google.com/github/Vishu-Gupta/MLProjects/blob/main/01%20Kaggle%20Projects/07%20Shift_Vibration_Analysis/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is analysis of a Kaggle Dataset.

https://www.kaggle.com/jishnukoliyadan/vibration-analysis-on-rotating-shaft

This dataset is intended to train and evaluate damage detection in rotary machines

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [5]:
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/

In [6]:
!kaggle datasets download -d jishnukoliyadan/vibration-analysis-on-rotating-shaft

100% 2.62G/2.62G [01:06<00:00, 60.4MB/s]
100% 2.62G/2.62G [01:06<00:00, 42.4MB/s]


In [7]:
!unzip vibration-analysis-on-rotating-shaft.zip

Archive:  vibration-analysis-on-rotating-shaft.zip
  inflating: 0D.csv                  
  inflating: 0E.csv                  
  inflating: 1D.csv                  
  inflating: 1E.csv                  
  inflating: 2D.csv                  
  inflating: 2E.csv                  
  inflating: 3D.csv                  
  inflating: 3E.csv                  
  inflating: 4D.csv                  
  inflating: 4E.csv                  


In [8]:
!rm vibration-analysis-on-rotating-shaft.zip

Installing Java Runtime ENV and Pyspark

In [9]:
# Installing spark 
!apt-get install openjdk-8-jre
!apt-get install scala
!pip install py4j
!wget -q https://downloads.apache.org/spark/spark-2.4.8/spark-2.4.8-bin-hadoop2.7.tgz
!tar xf spark-2.4.8-bin-hadoop2.7.tgz
!pip install -q findspark

# Installing databricks packages
!wget -q https://github.com/databricks/spark-deep-learning/archive/refs/tags/v1.5.0.zip 
!unzip v1.5.0.zip
!mv spark-deep-learning-1.5.0 databricks:spark-deep-learning:1.5.0-spark2.4-s_2.11

# Clearing unnecessary space
!rm -r *.tgz *.zip sample_data
!ls

# Setting up environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.8-bin-hadoop2.7"

SUBMIT_ARGS = "--packages databricks:spark-deep-learning:1.5.0-spark2.4-s_2.11 pyspark-shell"
os.environ["PYSPARK_SUBMIT_ARGS"] = SUBMIT_ARGS

# Importing and initating spark
import findspark
findspark.init()

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-470
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java
  libatk-wrapper-java-jni libgail-common libgail18 libgtk2.0-0 libgtk2.0-bin
  libgtk2.0-common libxxf86dga1 openjdk-8-jre-headless x11-utils
Suggested packages:
  gvfs icedtea-8-plugin libnss-mdns fonts-ipafont-gothic fonts-ipafont-mincho
  fonts-wqy-microhei fonts-wqy-zenhei fonts-indic mesa-utils
The following NEW packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java
  libatk-wrapper-java-jni libgail-common libgail18 libgtk2.0-0 libgtk2.0-bin
  libgtk2.0-common libxxf86dga1 openjdk-8-jre openjdk-8-jre-headless x11-utils
0 upgraded, 13 newly installed, 0 to remove and 39 not upgraded.
Need to get 33.6 MB of archive

In [12]:
## Spark Session setup
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Vibration Analysis').config('spark.executor.memory','10G').getOrCreate()

In [17]:
spark.sparkContext.getConf().get('spark.executor.memory')

'10G'

In [18]:
#loading the base dataset

base = spark.read.csv('0D.csv',inferSchema=True,header=True)

In [19]:
def shape(self):
  return (self.count(),len(self.columns))

In [20]:
shape(base)

(26423295, 5)

In [21]:
base.describe().show()

+-------+-----------------+------------------+--------------------+--------------------+--------------------+
|summary|             V_in|      Measured_RPM|         Vibration_1|         Vibration_2|         Vibration_3|
+-------+-----------------+------------------+--------------------+--------------------+--------------------+
|  count|         26423295|          26423295|            26423295|            26423295|            26423295|
|   mean|5.992249414760898|-35723.05444027922|0.001981370322707362|0.002713845821044...|0.004202591440439503|
| stddev|2.329659918182234|2987904.8204619107| 0.06284234971669785|  0.0868200929108851| 0.06599321125494297|
|    min|              0.0|            -2.4E8|         -0.10674953|         -0.20672441|        -0.036530495|
|    max|             10.0|         2376.6849|           7.8056097|           8.7808156|           7.7906823|
+-------+-----------------+------------------+--------------------+--------------------+--------------------+



In [22]:
base.where(base['Measured_RPM']==2376.6849).show(20)

+----+------------+-------------+-------------+-------------+
|V_in|Measured_RPM|  Vibration_1|  Vibration_2|  Vibration_3|
+----+------------+-------------+-------------+-------------+
|10.0|   2376.6849|  0.021424294| -0.020953417| 0.0043606758|
|10.0|   2376.6849|  -0.01521349|-0.0063800812| 0.0057315826|
|10.0|   2376.6849| -0.020757914|  0.017174482|  0.010704994|
|10.0|   2376.6849|  0.012702942| -0.010801554| 0.0014984608|
|10.0|   2376.6849| 0.0066196918|  -0.01229167| 0.0055921078|
|10.0|   2376.6849|  0.003554821|  0.044872761|-0.0018191338|
|10.0|   2376.6849|  0.027266741| 9.5129013E-4|-0.0027191639|
|10.0|   2376.6849| 0.0029671192| -0.009791851| 0.0052845478|
|10.0|   2376.6849| 0.0018966198|   0.01383543| 0.0036692619|
|10.0|   2376.6849|  -0.01432538|-0.0054216385|  0.010595322|
|10.0|   2376.6849| -0.013591051|-0.0025844574| 0.0066590309|
|10.0|   2376.6849| 0.0035464764| -0.028500557| 0.0084924698|
|10.0|   2376.6849|-0.0083315372|  0.027165413| 0.0034332275|
|10.0|  

In [24]:
base.where(base['Measured_RPM']==2376.6849).describe().show()

+-------+----+------------------+--------------------+--------------------+--------------------+
|summary|V_in|      Measured_RPM|         Vibration_1|         Vibration_2|         Vibration_3|
+-------+----+------------------+--------------------+--------------------+--------------------+
|  count|4096|              4096|                4096|                4096|                4096|
|   mean|10.0|2376.6849000000393|6.991619061643793E-4|6.626034193645498E-4|0.002716624296665502|
| stddev| 0.0|               0.0|0.015402569602379335|0.020645514196698987|0.005392896474520...|
|    min|10.0|         2376.6849|        -0.062990189|          -0.1009047|        -0.027856827|
|    max|10.0|         2376.6849|          0.10103703|           0.1103127|          0.02515316|
+-------+----+------------------+--------------------+--------------------+--------------------+



In [26]:
max_load = spark.read.csv('4D.csv',inferSchema=True,header=True)
max_load.describe().show()

+-------+-----------------+-------------------+--------------------+--------------------+--------------------+
|summary|             V_in|       Measured_RPM|         Vibration_1|         Vibration_2|         Vibration_3|
+-------+-----------------+-------------------+--------------------+--------------------+--------------------+
|  count|         26390527|           26390527|            26390527|            26390527|            26390527|
|   mean|5.997206118686653|-35769.199439385186|0.001139313278119...|0.002412792281254...|0.003749251604302...|
| stddev|2.326851954760891| 2989758.9354947447|0.040871358940044185| 0.08043873017025994|0.039561479999226774|
|    min|              0.0|             -2.4E8|         -0.12718558|         -0.20472765|        -0.041083097|
|    max|             10.0|          3830.5617|           6.8336797|           8.4695673|           6.5487444|
+-------+-----------------+-------------------+--------------------+--------------------+--------------------+



In [27]:
shape(max_load)

(26390527, 5)